# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('../data_files/sp_500_stocks.csv')
print(type(stocks))
print(stocks)

<class 'pandas.core.frame.DataFrame'>
    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets_api import IEX_CLOUD_API_TOKEN
print(IEX_CLOUD_API_TOKEN)

Tpk_059b97af715d417d9f49f50b51b1c448


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud: https://iexcloud.io/docs/api/

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
field = 'price'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 66343866, 'calculationPrice': 'close', 'change': 2.6, 'changePercent': 0.01544, 'close': 0, 'closeSource': 'faoiflic', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 171, 'iexCloseTime': 1681654729462, 'iexLastUpdated': 1714140180321, 'iexMarketPercent': 0.000663717910460642, 'iexOpen': 171.3, 'iexOpenTime': 1735659079008, 'iexRealtimePrice': 174.8, 'iexRealtimeSize': 51, 'iexVolume': 154, 'lastTradeTime': 1672784787443, 'latestPrice': 174.29, 'latestSource': 'Close', 'latestTime': 'August 25, 2022', 'latestUpdate': 1691869290943, 'latestVolume': None, 'low': 0, 'lowSource': 'IrempelXEc iita  er', 'lowTime': 1701436489993, 'marketCap': 2787596398582, 'oddLotDelayedPrice

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
print(f"Price: ${price}, with a Market Cap of: ${market_cap / (10**12)}T")

Price: $174.29, with a Market Cap of: $2.787596398582T


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ], index= my_columns),
    ignore_index=True
)

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/544378464.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,174.29,2787596398582,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ], index=my_columns),
        ignore_index=True)
    
final_dataframe

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/3479882798.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/3479882798.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/3479882798.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/3479882798.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.33,41960531638,N/A
1,AAL,15.11,9523712977,N/A
2,AAP,184.59,11210886877,N/A
3,AAPL,171.34,2838847614574,N/A
4,ABBV,139.94,246941547500,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunk(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [10]:
symbol_groups = list(chunk(stocks['Ticker'], 100))
symbol_strings = []
for group in symbol_groups:
    symbol_strings.append(','.join(group))

#print(symbol_strings)
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_call_api = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_call_api).json()
    for symbol in symbol_string.split(','):
        if symbol in data.keys():
            final_dataframe = final_dataframe.append(
                pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ], index=my_columns), ignore_index=True)
        else:
            print(f'Skipping: {symbol}...')

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

Skipping: DISCA...


/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

Skipping: HFC...


/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

Skipping: VIAC...
Skipping: WLTW...


/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/tl/b31rph5s7qz2n1_vbg0b71b40000gn/T/ipykernel_34975/2864599777.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.56,41031688053,N/A
1,AAL,14.58,9709840553,N/A
2,AAP,189.87,11348311704,N/A
3,AAPL,176.69,2775326976282,N/A
4,ABBV,139.38,256923890011,N/A
...,...,...,...,...
496,YUM,118.47,33764076317,N/A
497,ZBH,116.27,23930918263,N/A
498,ZBRA,344.47,17194290672,N/A
499,ZION,57.82,8676327919,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
portfolio_size = None
invalid_val = True
while portfolio_size is None or invalid_val:
    portfolio_size = input('How big is your portfolio? ')
    try:
        val = float(portfolio_size)
        invalid_val = False
    except:
        print('Please enter a Number (Int or Float).')

How big is your portfolio? 10000


In [13]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

In [14]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.56,41031688053,0
1,AAL,14.58,9709840553,1
2,AAP,189.87,11348311704,0
3,AAPL,176.69,2775326976282,0
4,ABBV,139.38,256923890011,0
...,...,...,...,...
496,YUM,118.47,33764076317,0
497,ZBH,116.27,23930918263,0
498,ZBRA,344.47,17194290672,0
499,ZION,57.82,8676327919,0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [37]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [38]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color, 
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color, 
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color, 
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [28]:
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [39]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()